<a href="https://colab.research.google.com/github/DaAnMaGi/PI_ML_OPS/blob/main/consultas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Descarga de librerías necesarias

In [1]:
import pandas as pd

# Carga de los archivos

In [97]:
# Se obtienen las rutas
games_ruta = "/content/games_genres_c.json.gz"
items_ruta = "/content/items_hours.json.gz"
reviews_ruta = "/content/reviews_final.json.gz"

In [98]:
# Se obtienen los dataframes
games = pd.read_json(games_ruta,compression="gzip")
items = pd.read_json(items_ruta,compression="gzip")
reviews = pd.read_json(reviews_ruta,compression="gzip")

# PlayTimeGenre
def PlayTimeGenre( genero : str ): Debe devolver año con mas horas jugadas para dicho género. Ejemplo de retorno: {"Año de lanzamiento con más horas jugadas para Género X" : 2013}

In [51]:
# Se define la función playtimegenre
def PlayTimeGenre(genero=str):
  # Se obtiene un dataframe a partir de games
  # donde cada juego obtiene una fila por cada género que tenga.
  g = games.explode("genre")
  # Se filtra el dataframe para obtener únicamente los
  # juegos que tengan el género especificado.
  g = g[g["genre"] == genero]

  # Se filtra el dataframe de items para encontrar aquellos que cumplen
  # la condición del género especificado.
  i = items[items["id_game"].isin(g["id_game"])]

  # Se obtiene la suma de horas jugadas por año del conjunto de juegos filtrado.
  x = i.groupby("year")["playtime_forever"].sum()

  # Se obtiene el año con más horas jugadas y el número de horas jugadas.
  year = x.idxmax()
  hours = x.max()

  # Se imprime la respuesta.
  return print(f"Para el género {genero} el año con más horas fue:\n{year}\nCon un total de {hours} horas jugadas")

In [53]:
genero = "Adventure"
PlayTimeGenre(genero)

Para el género Adventure el año con más horas fue:
2006.0
Con un total de 417199674 horas jugadas


# UserForGenre
def UserForGenre( genero : str ): Debe devolver el usuario que acumula más horas jugadas para el género dado y una lista de la acumulación de horas jugadas por año. Ejemplo de retorno: {"Usuario con más horas jugadas para Género X" : us213ndjss09sdf, "Horas jugadas":[{Año: 2013, Horas: 203}, {Año: 2012, Horas: 100}, {Año: 2011, Horas: 23}]}

In [81]:
def UserForGenre(genero = str):
  # Se obtiene un dataframe a partir de games
  # donde cada juego obtiene una fila por cada género que tenga.
  g = games.explode("genre")
  # Se filtra el dataframe para obtener únicamente los
  # juegos que tengan el género especificado.
  g = g[g["genre"] == genero]

  # Se filtra el dataframe de items para encontrar aquellos que cumplen
  # la condición del género especificado.
  i = items[items["id_game"].isin(g["id_game"])]

  # Se obtiene la información de horas jugadas por jugador.
  x = i.groupby(["user_id","year"])["playtime_forever"].sum()

  # Se obtiene el jugador con más horas jugadas.
  player = x.idxmax()[0]

  # Se obtiene la información para el jugador identificado.
  player_data = x.loc[player]

  # Se imprime el jugador con más horas jugadas para el género identificado.
  print(f"El ID del jugador con más horas jugadas para el género {genero} es:\n{player}")
  # Se imprimen el número de horas jugadas por año.
  for year,playtime in player_data.items():
    print(f"Para el año {year} jugó {playtime} horas")

In [87]:
genero = "Free to Play"

UserForGenre(genero)

El ID del jugador con más horas jugadas para el género Free to Play es:
shinomegami
Para el año 2010.0 jugó 1934 horas
Para el año 2012.0 jugó 2043 horas
Para el año 2015.0 jugó 530882 horas


# UsersRecommend
def UsersRecommend( año : int ): Devuelve el top 3 de juegos MÁS recomendados por usuarios para el año dado. (reviews.recommend = True y comentarios positivos/neutrales) Ejemplo de retorno: [{"Puesto 1" : X}, {"Puesto 2" : Y},{"Puesto 3" : Z}]

In [165]:
# Se define la función.
def UserRecommend(year):
  # Se obtienen las reviews para el año señalado
  r = reviews[reviews["review_year"] == year]

  # Se obtiene la sumatoria de recomendaciones por juego
  # para el año selecionado
  m = r.groupby("id_game")["recommend"].sum().sort_values(ascending=False)

  # Se obtienen los tres juegos con mayor número de recomendaciones.
  most_recommend = m[:3]

  # Se obtiene la respuesta
  print(f"Para el año {year} los juegos más recomendados fueron:")

  puesto = 1 # Variable que indica el lugar en el podio.

  for game_id, recom in most_recommend.items():
      # Se obtiene el nombre del juego usando el game_id.
      name = games.loc[games['id_game'] == game_id, 'name'].values[0]
      # Se imprimen los puestos
      print(f"En el puesto #{puesto}, el juego: {name}, con {recom} recomendaciones.")
      puesto += 1

In [178]:
year = 2012
UserRecommend(year)

Para el año 2012 los juegos más recomendados fueron:
En el puesto #1, el juego: Team Fortress 2, con 206 recomendaciones.
En el puesto #2, el juego: Garry's Mod, con 33 recomendaciones.
En el puesto #3, el juego: Terraria, con 30 recomendaciones.


# UsersWorstDeveloper

def UsersWorstDeveloper( año : int ): Devuelve el top 3 de desarrolladoras con juegos MENOS recomendados por usuarios para el año dado. (reviews.recommend = False y comentarios negativos) Ejemplo de retorno: [{"Puesto 1" : X}, {"Puesto 2" : Y},{"Puesto 3" : Z}]

In [196]:
# Se define la función.
def UserWorstDeveloper(year):
  # Se obtienen las reviews para el año señalado
  r = reviews[reviews["review_year"] == year]

  # Se obtienen las desarrolladoras de los juegos
  # con reviews para el año selecionado
  g = games[["id_game","developer"]]
  m = r.merge(g,on="id_game")

  # Se obtienen el número de recomendaciones para cada desarrollador.
  x = m.groupby("developer")["recommend"].sum().sort_values(ascending=True)

  # Se obtienen las tres desarrolladoras con menor número de recomendaciones.
  least_recommend = x[:3]

  # Se obtiene la respuesta
  print(f"Para el año {year} las desarrolladoras menos recomendadas fueron:")

  puesto = 1 # Variable que indica el lugar en el podio.

  for desarrolladora, recom in least_recommend.items():
    # Se imprimen los puestos
    print(f"En el puesto #{puesto}, la desarrolladora: {desarrolladora}, con {recom} recomendaciones.")
    puesto += 1


In [202]:
year = 2010
UserWorstDeveloper(year)

Para el año 2010 las desarrolladoras menos recomendadas fueron:
En el puesto #1, la desarrolladora: 1C Company, con 1 recomendaciones.
En el puesto #2, la desarrolladora: Ubisoft Montreal, con 1 recomendaciones.
En el puesto #3, la desarrolladora: The Creative Assembly,Feral Interactive (Mac),Feral Interactive (Linux), con 1 recomendaciones.


In [193]:
most_recommend

developer
Puny Human          0
Moment Studio       0
DiggerWorld Ltd.    0
Name: recommend, dtype: int64

# sentiment_analysis
def sentiment_analysis( empresa desarrolladora : str ): Según la empresa desarrolladora, se devuelve un diccionario con el nombre de la desarrolladora como llave y una lista con la cantidad total de registros de reseñas de usuarios que se encuentren categorizados con un análisis de sentimiento como valor. Ejemplo de retorno: {'Valve' : [Negative = 182, Neutral = 120, Positive = 278]}